# NLP Tasks with Transformers (Huggingface)

## • Sentiment analysis
## • Text generation
## • Name entity recognition (NER)
## • Question answering
## • Filling masked text
## • Summarization
## • Translation
## • Feature extraction

### • Sentiment analysis : 

In [4]:
from transformers import pipeline

# I use model "nlptown/bert-base-multilingual-uncased-sentiment" for sentiment analysis on 
# product reviews in six languages: English, Dutch, German, French, Spanish and Italian. 
# It predicts the sentiment of the review as a number of stars (between 1 and 5).

Sentiment_analysis = pipeline("sentiment-analysis",model="nlptown/bert-base-multilingual-uncased-sentiment")

result_ENG = Sentiment_analysis("I hate you")[0]
print(f"label: {result_ENG['label']}, with score: {round(result_ENG['score'], 4)}")

result_ENG = Sentiment_analysis("I love you")[0]
print(f"label: {result_ENG['label']}, with score: {round(result_ENG['score'], 4)}")

result_FR = Sentiment_analysis("je te aime maman")[0]
print(f"label: {result_FR['label']}, with score: {round(result_FR['score'], 4)}")

result_FR = Sentiment_analysis("je te déteste")[0]
print(f"label: {result_FR['label']}, with score: {round(result_FR['score'], 4)}")




label: 1 star, with score: 0.6346
label: 5 stars, with score: 0.8547
label: 5 stars, with score: 0.4693
label: 1 star, with score: 0.4572


### • Text generation :

In [7]:
from transformers import pipeline

#Text English : 

text_generator_ENG = pipeline("text-generation",model="xlnet-base-cased")
print(text_generator_ENG("i'm okay but", max_length=50, do_sample=False))

#Text Arabic : 

text_generator_AR = pipeline("text-generation",model="mofawzy/gpt2-arabic-sentence-generator")
print(text_generator_AR("كيف", max_length=50, do_sample=False))


[{'generated_text': "i'm okay but i don't know what to do with this. I don't know what to do with this. I don't know what to do with this. I don't know what to do with this"}]


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': 'كيف يكون هناك بعض المقالات التي لم تعجبني"\n"كتاب خفيف . خفيف . خفيف . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .'}]


### • Name entity recognition (NER) :

In [14]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch 


# 1 - Name entity recognition NER (English) :

model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

label_list = [
        "O",       # Outside of a named entity
        "B-MISC",  # Beginning of a miscellaneous entity right after another miscellaneous entity
        "I-MISC",  # Miscellaneous entity
        "B-PER",   # Beginning of a person's name right after another person's name
        "I-PER",   # Person's name
        "B-ORG",   # Beginning of an organisation right after another organisation
        "I-ORG",   # Organisation
        "B-LOC",   # Beginning of a location right after another location
        "I-LOC"    # Location
 ]

sequence = "My name is Mohamed and I live in Berlin."

# Bit of a hack to get the tokens with the special tokens

tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
inputs = tokenizer.encode(sequence, return_tensors="pt")

outputs = model(inputs).logits
predictions = torch.argmax(outputs, dim=2)

print("""Name entity recognition NER (English) :\n""")
print([(token, label_list[prediction]) for token, prediction in zip(tokens, predictions[0].numpy())])

# 2 - Name entity recognition NER (Arabic) :
 
model = AutoModelForTokenClassification.from_pretrained("hatmimoha/arabic-ner")
tokenizer = AutoTokenizer.from_pretrained("hatmimoha/arabic-ner")

label_list = [
    "0", "B-PERSON",
    "1", "I-PERSON",
    "2", "B-ORGANIZATION",
    "3", "I-ORGANIZATION",
    "4", "B-LOCATION",
    "5", "I-LOCATION",
    "6", "B-DATE",
    "7", "I-DATE",
    "8", "B-COMPETITION",
    "9", "I-COMPETITION",
    "10", "B-PRIZE",
    "11", "I-PRIZE",
    "12", "O",
    "13", "B-PRODUCT",
    "14", "I-PRODUCT",
    "15", "B-EVENT",
    "16", "I-EVENT",
    "17", "B-DISEASE",
    "18", "I-DISEASE",
 ]

sequence = "إسمي محمد وأسكن في برلين"

# Bit of a hack to get the tokens with the special tokens

tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
inputs = tokenizer.encode(sequence, return_tensors="pt")

outputs = model(inputs).logits
predictions = torch.argmax(outputs, dim=2)

print("""\n Name entity recognition NER (Arabic) :\n""")
print([(token, label_list[prediction]) for token, prediction in zip(tokens, predictions[0].numpy())])

Name entity recognition NER (English) :

[('[CLS]', 'O'), ('My', 'O'), ('name', 'O'), ('is', 'O'), ('Mohamed', 'I-PER'), ('and', 'O'), ('I', 'O'), ('live', 'O'), ('in', 'O'), ('Berlin', 'I-LOC'), ('.', 'O'), ('[SEP]', 'O')]

 Name entity recognition NER (Arabic) :

[('[CLS]', '6'), ('[UNK]', '6'), ('محمد', '0'), ('[UNK]', 'B-PERSON'), ('في', '6'), ('برلين', '2'), ('[SEP]', '6')]


### • Question answering :

In [16]:
from transformers import pipeline

Question_answering = pipeline("question-answering")

context = r""" Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a
question answering dataset is the SQuAD dataset, which is entirely based on that task. If you would like to fine-tune
a model on a SQuAD task, you may leverage the examples/question-answering/run_squad.py script."""

result = Question_answering(question="What is extractive question answering?", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

result2 = Question_answering(question="What is a good example of a question answering dataset?", context=context)
print(f"Answer: '{result2['answer']}', score: {round(result2['score'], 4)}, start: {result2['start']}, end: {result2['end']}")


Answer: 'the task of extracting an answer from a text given a question', score: 0.6226, start: 34, end: 95
Answer: 'SQuAD dataset', score: 0.5053, start: 147, end: 160


### • Filling masked text :

In [8]:
from transformers import AutoModelWithLMHead, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
model = AutoModelWithLMHead.from_pretrained("distilbert-base-cased")

sequence = f"Distilled models are smaller than the models they mimic. Using them instead of the large versions would help {tokenizer.mask_token} our carbon footprint."

input = tokenizer.encode(sequence, return_tensors="pt")
mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]

token_logits = model(input).logits
mask_token_logits = token_logits[0, mask_token_index, :]

top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

#Top 5 Tokens : 

for token in top_5_tokens:
   print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

c:\users\zakaria\appdata\local\programs\python\python39\lib\site-packages\transformers\models\auto\modeling_auto.py:843: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Downloading:   0%|          | 0.00/263M [00:00<?, ?B/s]

Distilled models are smaller than the models they mimic. Using them instead of the large versions would help reduce our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help increase our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help decrease our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help offset our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help improve our carbon footprint.


### • Summarization : 

In [11]:
from transformers import pipeline

summarizer = pipeline("summarization")

ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18."""

print(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))

Downloading:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

[{'summary_text': ' Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, nine of them between 1999 and 2002 . She is believed to still be married to four men .'}]


### • Translation : 

In [2]:
from transformers import MarianTokenizer, MarianMTModel

mname = "marefa-nlp/marefa-mt-en-ar"

tokenizer = MarianTokenizer.from_pretrained(mname)
model = MarianMTModel.from_pretrained(mname)

# English Sample Text
inputs = "Ahmad went to school"

translated_tokens = model.generate(**tokenizer.prepare_seq2seq_batch([inputs], return_tensors="pt"))
translated_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translated_tokens]

# translated Arabic Text
print(translated_text)

['التحق أحمد بالمدرسة']


### • Feature extraction :

In [17]:
from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/distilbert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/distilbert-base-nli-mean-tokens')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, max pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

print("Sentence embeddings: \n")
print(sentence_embeddings)

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/265M [00:00<?, ?B/s]

Sentence embeddings: 

tensor([[-0.6313,  0.0280, -0.4024,  ...,  0.0632, -0.1281, -0.5619],
        [-0.5223, -0.2834,  0.7265,  ..., -0.2099, -0.0241, -0.2587]])


## Reference :


#### - https://huggingface.co/transformers/v4.1.1/task_summary.html
#### - https://huggingface.co/models

### Feature extraction :

#### - https://huggingface.co/sentence-transformers/distilbert-base-nli-mean-tokens

### Translation :

#### - https://huggingface.co/marefa-nlp/marefa-mt-en-ar

### Filling masked text :

#### - https://huggingface.co/distilbert-base-cased

### Name entity recognition (NER) :

#### - EN : - https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english
#### - AR : - https://huggingface.co/hatmimoha/arabic-ner 

### Text generation : 

#### - EN : - https://huggingface.co/xlnet-base-cased
#### - AR : - https://huggingface.co/mofawzy/gpt2-arabic-sentence-generator

### Sentiment analysis : 

#### - https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment
